In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
# Mostrar DataFrames sin saltos de línea
pd.set_option('display.expand_frame_repr', False)
import ast



In [2]:
# Loading csv with results
df = pd.read_csv("results_top_trials_v2.csv", sep=";", na_values="NaN")
df.head()

,Unnamed: 0,experiment_id,trial_index,dino_model,normalization,scaler,dim_red,reduction_parameters,clustering,n_clusters,...,penalty_range,noise_not_noise,classification_lvl,lvlm,prompt,eval_method,best_score,homogeneity_global,entropy_global,quality_metric
0,0,1,355,small,True,NaN,umap,"{'metric': 'cosine', 'n_components': 2, 'n_nei...",hdbscan,156,...,NaN,"{-1: 1923, 1: 3143}",3,llava1-6_7b,1,silhouette,0.725811,0.803054,0.657708,1.220988
1,1,1,361,small,True,NaN,umap,"{'metric': 'cosine', 'n_components': 2, 'n_nei...",hdbscan,155,...,NaN,"{-1: 2084, 1: 2982}",3,llava1-6_7b,1,silhouette,0.724763,0.803054,0.657708,1.220988
2,2,1,13,small,True,NaN,umap,"{'metric': 'euclidean', 'n_components': 2, 'n_...",hdbscan,162,...,NaN,"{-1: 1757, 1: 3309}",3,llava1-6_7b,1,silhouette,0.721921,0.803054,0.657708,1.220988
3,3,1,7,small,True,NaN,umap,"{'metric': 'euclidean', 'n_components': 2, 'n_...",hdbscan,233,...,NaN,"{-1: 1769, 1: 3297}",3,llava1-6_7b,1,silhouette,0.719861,0.803054,0.657708,1.220988
4,4,1,19,small,True,NaN,umap,"{'metric': 'euclidean', 'n_components': 2, 'n_...",hdbscan,130,...,NaN,"{-1: 1895, 1: 3171}",3,llava1-6_7b,1,silhouette,0.717324,0.803054,0.657708,1.220988


In [3]:
import pandas as pd
import ast

def filter_results(df_res):
    # Crear una copia del DataFrame
    df_res = df_res.copy()

    # Manejar valores NaN antes de aplicar la conversión
    df_res['reduction_parameters'] = df_res['reduction_parameters'].fillna('{"n_components":"all"}')
    df_res['scaler'] = df_res['scaler'].fillna('None')

    # Extraer 'n_components'
    df_res['n_components'] = df_res['reduction_parameters'].apply(
        lambda x: ast.literal_eval(x).get('n_components', 'all')
    )

    # Calcular y agregar 'noise_ratio'
    df_res['noise_ratio'] = df_res['noise_not_noise'].apply(
        lambda x: round(ast.literal_eval(x).get(-1, 0) / 5066, 3)
    )

    df_res['best_score'] = round(df_res['best_score'],3)

    # Eliminar la columna de parámetros reducidos
    df_res = df_res.drop(["reduction_parameters"], axis=1)

    # Seleccionar los 10 mejores ensayos únicos de cada algoritmo
    top10_unique_trials = df_res.sort_values(
        by="best_score", ascending=False
    ).groupby('clustering').head(10)

    # Ordenar los resultados
    ascending =  False if df_res['eval_method'].unique() in ("silhouette","silhouette_noise") else True
    top10_sorted = top10_unique_trials.sort_values(by="best_score", ascending=ascending)

    # Devolver el DataFrame actualizado
    return top10_sorted[[
        "experiment_id", "trial_index", "clustering","normalization", "scaler",
        "dim_red", "n_components", "noise_ratio", "n_clusters", "eval_method", "best_score"
    ]]



In [4]:
import pandas as pd
import ast

def filter_results_less_noise_ratio(df_res):
    # Crear una copia del DataFrame
    df_res = df_res.copy()

    # Manejar valores NaN antes de aplicar la conversión
    df_res['reduction_parameters'] = df_res['reduction_parameters'].fillna('{"n_components":"all"}')
    df_res['scaler'] = df_res['scaler'].fillna('None')

    # Extraer 'n_components'
    df_res['n_components'] = df_res['reduction_parameters'].apply(
        lambda x: ast.literal_eval(x).get('n_components', 'all')
    )

    # Calcular y agregar 'noise_ratio'
    df_res['noise_ratio'] = df_res['noise_not_noise'].apply(
        lambda x: round(ast.literal_eval(x).get(-1, 0) / 5066, 3)
    )

    df_res['best_score'] = round(df_res['best_score'], 3)

    # Eliminar la columna de parámetros reducidos
    df_res = df_res.drop(["reduction_parameters"], axis=1)

    # Seleccionar los 10 mejores ensayos únicos de cada algoritmo
    top10_unique_trials = df_res.sort_values(
        by=["noise_ratio", "best_score"], 
        ascending=[True, False]  # Menor noise_ratio, mejor best_score
    ).groupby('clustering').head(10)

    # Devolver el DataFrame actualizado
    return top10_unique_trials[[
        "experiment_id", "trial_index", "clustering", "normalization", "scaler", 
        "dim_red", "n_components", "noise_ratio", "n_clusters", "eval_method", "best_score"
    ]]


In [37]:
df_res = df[(df["eval_method"] == "davies_bouldin") & 
                        (df["n_clusters"] > 2) &
                        (df["clustering"] == "hdbscan") &
                          (df["dino_model"] == "base")][["experiment_id", "trial_index", "clustering","normalization", "scaler","dim_red", 
                                                         "reduction_parameters","noise_not_noise", "n_clusters", "eval_method", "best_score"]].drop_duplicates()

#print(filter_results(df_res))
print(filter_results_less_noise_ratio(df_res))


     experiment_id  trial_index clustering  normalization    scaler dim_red  n_components  noise_ratio  n_clusters     eval_method  best_score
819             29          140    hdbscan           True      None    umap             5        0.000          46  davies_bouldin       0.465
577             21           80    hdbscan           True  standard    umap             2        0.000          48  davies_bouldin       0.421
487             18           80    hdbscan           True      None    umap             2        0.000          54  davies_bouldin       0.406
817             29           80    hdbscan           True      None    umap             2        0.000          54  davies_bouldin       0.406
816             29            0    hdbscan           True      None    umap             2        0.000          51  davies_bouldin       0.382
576             21            0    hdbscan           True  standard    umap             2        0.000          67  davies_bouldin       0.357

In [20]:
import pandas as pd
import ast

def filter_results_best_homegeneity_global(df_res, with_noise=False):
    # Crear una copia del DataFrame
    df_res = df_res.copy()

    # Manejar valores NaN antes de aplicar la conversión
    df_res['reduction_parameters'] = df_res['reduction_parameters'].fillna('{"n_components":"all"}')
    df_res['scaler'] = df_res['scaler'].fillna('None')

    # Extraer 'n_components'
    df_res['n_components'] = df_res['reduction_parameters'].apply(
        lambda x: ast.literal_eval(x).get('n_components', 'all')
    )

    # Calcular y agregar 'noise_ratio'
    df_res['noise_ratio'] = df_res['noise_not_noise'].apply(
        lambda x: round(ast.literal_eval(x).get(-1, 0) / 5066, 3)
    )

    # Redondear valores relevantes
    df_res['best_score'] = round(df_res['best_score'], 3)
    df_res['homogeneity_global'] = round(df_res['homogeneity_global'], 3)
    df_res['quality_metric'] = round(df_res['quality_metric'], 3)

    df_res['quality_metric_noise_ratio'] = df_res['quality_metric']/df_res['noise_ratio']

    # Eliminar la columna de parámetros reducidos
    df_res = df_res.drop(["reduction_parameters"], axis=1)

    # Ordenar el DataFrame por quality_metric con menor noise_ratio
    df_sorted = df_res.sort_values(
        by=["noise_ratio", "quality_metric"], 
        ascending=[True, False]
    )

    # Obtener los 5 mejores experimentos
    top5_experiments = df_sorted.groupby('experiment_id').head(1)['experiment_id'].unique()
    print(top5_experiments)
    # Filtrar los mejores experimentos
    top_experiment_trials = df_sorted[df_sorted['experiment_id'].isin(top5_experiments)]

    # Obtener los 3 mejores ensayos de cada experimento
    top3_trials_per_experiment = top_experiment_trials.groupby('experiment_id').head(3)



    # Devolver el DataFrame actualizado
    return top3_trials_per_experiment[[
        "experiment_id", "best_trial_index", "clustering", "dino_model", 
        "dim_red", "n_components", "noise_ratio", "n_clusters", 
        "eval_method", "best_score", "lvlm", "prompt", "quality_metric"
    ]]


In [24]:
## TOP 5 KMEANS SILHOUETTE SMALL

# Loading csv with results
df = pd.read_csv("results.csv", sep=";", na_values="NaN")
df.head()

with_noise = False
df_res = df[(df["n_clusters"] > 2) &
            (df["lvlm"] != "llava1-5_7b") &
            (df["dim_red"].notna()) &
             (df["clustering"] == "hdbscan") &
             (df["n_clusters"] > 100) ][
                              ["experiment_id", "best_trial_index", "clustering","normalization", "scaler","dim_red", 
                              "reduction_parameters","noise_not_noise", "n_clusters", "eval_method", "dino_model",
                              "lvlm","prompt","homogeneity_global","quality_metric","best_score"]].drop_duplicates()

#print(filter_results(df_res))
#print(filter_results_less_noise_ratio(df_res))
with pd.option_context('display.max_rows', None):
    df_def = filter_results_best_homegeneity_global(df_res, False).head(10)
    df_def = df_def[df_def["score"]].sort_values(by="quality_metric", ascending=False, inplace=True)
    print(df_def)


[19 28 22 66 65 47 46 43 42 45 44 64  5 12 41  1 11 40  9  6 13  8 16  3
  4 62]
     experiment_id  best_trial_index clustering dino_model dim_red  n_components  noise_ratio  n_clusters       eval_method  best_score          lvlm  prompt  quality_metric
163             28                60    hdbscan       base    umap             5        0.091         262        silhouette       0.640   llava1-6_7b       2           1.494
391             66                45    hdbscan       base    umap            11        0.146         171      davies_noise       0.548   llava1-6_7b       2           1.432
127             22                60    hdbscan       base    umap             5        0.101         214        silhouette       0.642   llava1-6_7b       2           1.391
109             19                32    hdbscan       base    umap             4        0.089         213  silhouette_noise       0.620   llava1-6_7b       2           1.376
162             28                60    hdbscan  